In [ ]:
# Week 5 - Financial APIs // building a client

In [7]:
import requests 
import pandas as pd
import io
import matplotlib as plt
import numpy as np
from scipy import stats
import matplotlib.pyplot as plta

In [38]:
API_KEY = "J9C3GHZ237UWFTA2"

In [9]:
def alpha_vantage_equity(symb,API_KEY):
    link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={}&outputsize=full&apikey={}&datatype=csv'.format(symb,API_KEY)
    rq = requests.get(link)
    urlData = rq.content
    rawData = pd.read_csv(io.StringIO(urlData.decode('utf-8')))
    # returns encoded utf-8 CSV format but in a Pandas DataFrame
    return rawData

In [42]:
def build_table(symb,API_KEY):
    
    data = alpha_vantage_equity(symb,API_KEY)
    # in 1 year there are 252 trading days 
    year_data = data[0:252]
    frame = year_data.sort_values(by='timestamp')
    frame.set_index('timestamp',inplace=True)
    frame['Returns'] = frame['adjusted_close'].pct_change()
    r_timeseries = frame.Returns.dropna()

    # mean
    mean = np.mean(r_timeseries)
    # annualized return
    m_annual = ((1+mean)**252)-1


    # risk measures
    vol = (np.std(r_timeseries))**2
    v_annual = np.std(r_timeseries)*np.sqrt(252)

    risk_free_rate = 1.60
    # sharpe ratio   ASSET A has AR - 207% and a risk of 50% ---- RF ER=0 and risk of 0%
    sr = (m_annual-risk_free_rate)/(v_annual/np.sqrt(252))

    out = [m_annual,v_annual,sr]
    

    return out

In [44]:
build_table('FB',API_KEY)

[1.0363488296751693, 0.4025890968952986, -22.225353359599545]